$$\large{\mathbf{Instituto\ Superior\ de\ Engenharia\ de\ Lisboa}}$$

$$\large{\mathrm{Licenciatura\ em\ Engenharia\ Informática\ e\ Multimédia}}$$

$$\Large{\mathbf{Codificação\ de\ Sinais\ Multimédia}}$$

$$\normalsize{\mathbf{3º\ Trabalho\ Prático\\}}$$

In [1]:
import numpy as np
import math
import cv2
import os
import sys
import time
import string
import random
import threading
import traceback
import psutil
import humanize
import resource
!pip install GPUtil
import GPUtil as GPU
import matplotlib.pyplot as plt
from pprint import pprint
from enum import Enum, unique
from google.colab import drive, widgets

  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built GPUtil


In [2]:
class Notebook():
    """Holds notebook methods and variables."""
    ROOT = '/content/drive'
    PROJECT = ROOT + '/My Drive/CSM/isel_leim_csm_1819/projects/P03'
    LIBRARY = ROOT + '/My Drive/CSM/isel_leim_csm_1819/projects/lib'
    NOTEBOOK = PROJECT + '/notebooks/'
    RESOURCES = PROJECT + '/resources/raw/'
    OUTPUT = PROJECT + '/resources/processed/'
    GPU = GPU.getGPUs()
    PROCESS = psutil.Process(os.getpid())
    
    @classmethod
    def mount(cls):
        """Mounts the (Google) drive in this runtime, all files/folders will be available."""
        drive.mount("/content/drive", force_remount=True)
        for path in [cls.ROOT, cls.PROJECT, cls.LIBRARY, cls.NOTEBOOK, cls.RESOURCES, cls.OUTPUT]:
            try:
                assert os.path.isdir(path), 'Directory not found.'
                sys.path.insert(0, path) #To allow for direct imports
            except AssertionError:
                _, _, tb = sys.exc_info()
                tb_info = traceback.extract_tb(tb)
                filename, line, func, text = tb_info[-1]
                print('An error occurred on line {} in statement {}:'.format(line, text))
                sys.exit(0)
    
    @classmethod
    def memory(cls):
        """Checks the RAM resources available in this runtime, including peak usage stats."""
        print('Gen Available RAM: {} | Process Memory Size: {} | Peak Usage: {} MB'
            .format(
                humanize.naturalsize(psutil.virtual_memory().available), 
                humanize.naturalsize(cls.PROCESS.memory_info().rss),
                resource.getrusage(resource.RUSAGE_SELF).ru_maxrss/1000.))
        for g in cls.GPU:
            print('GPU Available RAM: {0:.1f} GB | Used: {1:.1f} GB | Util {2:0.0f}% | Total {3:.1f} GB'
                .format(
                    g.memoryFree/1000., g.memoryUsed/1000., g.memoryUtil/10., g.memoryTotal/1000.))
    
Notebook.mount()
Notebook.memory()

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
Gen Available RAM: 12.9 GB | Process Memory Size: 136.2 MB | Peak Usage: 133.004 MB
GPU Available RAM: 15.1 GB | Used: 0.0 GB | Util 0% | Total 15.1 GB


In [0]:
@unique
class FileType(Enum):
    """Lists type of input files as an integer."""
    TEXT, IMAGE, SOUND = range(3)

    @classmethod
    def get_name(cls, x) -> enumerate.__name__:
        """Returns type name from its value."""
        return [y for y in cls if x == y.value][0]

    @classmethod
    def get_value(cls, x) -> int:
        """Returns type value from its name."""
        return x.value

    @classmethod
    def get_bit_length(cls) -> int:
        """Checks max bit length needed to represent any file type."""
        return list(cls._value2member_map_)[-1].bit_length()

In [0]:
from abc import ABC, abstractmethod

class JPEG(ABC):
    """Abstract class parenting JPEG encoding/decoding configurations."""
    
    @abstractmethod
    def __init__(self, 
                 quality: float = 1,
                 chroma_sampling: bool = True,
                 chroma_type: tuple = (4, 2, 0),
                 block_shape: tuple = (8, 8)) -> None:
        self.__quality = quality
        self.__chroma_sampling = chroma_sampling
        self.__chroma_type = chroma_type
        self.__block_shape = block_shape   
    
    @property
    def quality(self) -> float:
        """Returns the preset quality factor."""
        return self.__quality
    
    @property
    def block_shape(self) -> tuple:
        """Returns the preset block shape."""
        return self.__block_shape

    @property
    def chroma_sampling(self) -> bool:
        """Returns the preset chroma sampling."""
        return self.__chroma_sampling
           
    @property
    def chroma_type(self) -> tuple:
        """Returns the preset chroma type."""
        return self.__chroma_type
    
    @quality.setter
    def quality(self, quality: float) -> None:
        """Resets the quality factor."""
        self.__quality = quality
        
    @block_shape.setter
    def block_shape(self, block_shape: tuple) -> None:
        """Resets the block_shape."""
        self.__block_shape = block_shape
    
    @chroma_sampling.setter
    def chroma_sampling(self, chroma_sampling: bool) -> None:
        """Resets the chroma sampling."""
        self.__chroma_sampling = chroma_sampling
        
    @chroma_type.setter
    def chroma_type(self, chroma_type: tuple) -> None:
        """Resets the chroma type."""
        self.__chroma_type = chroma_type
    
    LUMINANCE = (1,)
    
    CHROMINANCE = (2, 3)
    
    EOB = '1010'
     
    ############## Luminance Quantization Table ##############
    K1 = np.array([
                    [  17,  18,  24,  47,  99,  99,  99,  99],
                    [  18,  21,  26,  66,  99,  99,  99,  99],
                    [  24,  26,  56,  99,  99,  99,  99,  99],
                    [  47,  66,  99,  99,  99,  99,  99,  99],
                    [  99,  99,  99,  99,  99,  99,  99,  99],
                    [  99,  99,  99,  99,  99,  99,  99,  99],
                    [  99,  99,  99,  99,  99,  99,  99,  99],
                    [  99,  99,  99,  99,  99,  99,  99,  99],
                  ])
    
    ############## Chrominance Quantization Table ##############
    K2 = np.array([
                    [  16,  11,  10,  16,  24,  40,  51,  61 ],
                    [  12,  12,  14,  19,  26,  58,  60,  55 ],
                    [  14,  13,  16,  24,  40,  57,  69,  56 ],
                    [  14,  17,  22,  29,  51,  87,  80,  62 ],
                    [  18,  22,  37,  56,  68, 109, 103,  77 ],
                    [  24,  35,  55,  64,  81, 104, 113,  92 ],
                    [  49,  64,  78,  87, 103, 121, 120, 101 ],
                    [  72,  92,  95,  98, 112, 100, 103,  99 ]
                  ])
    
    ############## DC Coefficients Encoding Table ##############
    K3 = {
            0:  '00',
            1:  '010',
            2:  '011',
            3:  '100',
            4:  '101',
            5:  '110',
            6:  '1110',
            7:  '11110',
            8:  '111110',
            9:  '1111110',
            10: '11111110',
            11: '111111110',
          }
    
    ############## Zig-Zag Index Table ##############
    K4 = np.array([
                    [ 0,  1,  5,  6, 14, 15, 27, 28],
                    [ 2,  4,  7, 13, 16, 26, 29, 42],
                    [ 3,  8, 12, 17, 25, 30, 41, 43],
                    [ 9, 11, 18, 24, 31, 40, 44, 53],
                    [10, 19, 23, 32, 39, 45, 52, 54],
                    [20, 22, 33, 38, 46, 51, 55, 60],
                    [21, 34, 37, 47, 50, 56, 59, 61],
                    [35, 36, 48, 49, 57, 58, 62, 63],
                  ])
    
    ############## AC Coefficients Encoding Table ##############
    K5 = {
            # (0, 0): '1010',
            (0, 1):   '00',
            (0, 2):   '01',
            (0, 3):   '100',
            (0, 4):   '1011',
            (0, 5):   '11010',
            (0, 6):   '1111000',
            (0, 7):   '11111000',
            (0, 8):   '1111110110',
            (0, 9):   '1111111110000010',
            (0, 10):  '1111111110000011',
            (1, 1):   '1100',
            (1, 2):   '11011',
            (1, 3):   '1111001',
            (1, 4):   '111110110',
            (1, 5):   '11111110110',
            (1, 6):   '1111111110000100',
            (1, 7):   '1111111110000101',
            (1, 8):   '1111111110000110',
            (1, 9):   '1111111110000111',
            (1, 10):  '1111111110001000',
            (2, 1):   '11100',
            (2, 2):   '11111001',
            (2, 3):   '1111110111',
            (2, 4):   '111111110100',
            (2, 5):   '1111111110001001',
            (2, 6):   '1111111110001010',
            (2, 7):   '1111111110001011',
            (2, 8):   '1111111110001100',
            (2, 9):   '1111111110001101',
            (2, 10):  '1111111110001110',
            (3, 1):   '111010',
            (3, 2):   '111110111',
            (3, 3):   '111111110101',
            (3, 4):   '1111111110001111',
            (3, 5):   '1111111110010000',
            (3, 6):   '1111111110010001',
            (3, 7):   '1111111110010010',
            (3, 8):   '1111111110010011',
            (3, 9):   '1111111110010100',
            (3, 10):  '1111111110010101',
            (4, 1):   '111011',
            (4, 2):   '1111111000',
            (4, 3):   '1111111110010110',
            (4, 4):   '1111111110010111',
            (4, 5):   '1111111110011000',
            (4, 6):   '1111111110011001',
            (4, 7):   '1111111110011010',
            (4, 8):   '1111111110011011',
            (4, 9):   '1111111110011100',
            (4, 10):  '1111111110011101',
            (5, 1):   '1111010',
            (5, 2):   '11111110111',
            (5, 3):   '1111111110011110',
            (5, 4):   '1111111110011111',
            (5, 5):   '1111111110100000',
            (5, 6):   '1111111110100001',
            (5, 7):   '1111111110100010',
            (5, 8):   '1111111110100011',
            (5, 9):   '1111111110100100',
            (5, 10):  '1111111110100101',
            (6, 1):   '1111011',
            (6, 2):   '111111110110',
            (6, 3):   '1111111110100110',
            (6, 4):   '1111111110100111',
            (6, 5):   '1111111110101000',
            (6, 6):   '1111111110101001',
            (6, 7):   '1111111110101010',
            (6, 8):   '1111111110101011',
            (6, 9):   '1111111110101100',
            (6, 10):  '1111111110101101',
            (7, 1):   '11111010',
            (7, 2):   '111111110111',
            (7, 3):   '1111111110101110',
            (7, 4):   '1111111110101111',
            (7, 5):   '1111111110110000',
            (7, 6):   '1111111110110001',
            (7, 7):   '1111111110110010',
            (7, 8):   '1111111110110011',
            (7, 9):   '1111111110110100',
            (7, 10):  '1111111110110101',
            (8, 1):   '111111000',
            (8, 2):   '111111111000000',
            (8, 3):   '1111111110110110',
            (8, 4):   '1111111110110111',
            (8, 5):   '1111111110111000',
            (8, 6):   '1111111110111001',
            (8, 7):   '1111111110111010',
            (8, 8):   '1111111110111011',
            (8, 9):   '1111111110111100',
            (8, 10):  '1111111110111101',
            (9, 1):   '111111001',
            (9, 2):   '1111111110111110',
            (9, 3):   '1111111110111111',
            (9, 4):   '1111111111000000',
            (9, 5):   '1111111111000001',
            (9, 6):   '1111111111000010',
            (9, 7):   '1111111111000011',
            (9, 8):   '1111111111000100',
            (9, 9):   '1111111111000101',
            (9, 10):  '1111111111000110',
            (10, 1):  '111111010',
            (10, 2):  '1111111111000111',
            (10, 3):  '1111111111001000',
            (10, 4):  '1111111111001001',
            (10, 5):  '1111111111001010',
            (10, 6):  '1111111111001011',
            (10, 7):  '1111111111001100',
            (10, 8):  '1111111111001101',
            (10, 9):  '1111111111001110',
            (10, 10): '1111111111001111',
            (11, 1):  '1111111001',
            (11, 2):  '1111111111010000',
            (11, 3):  '1111111111010001',
            (11, 4):  '1111111111010010',
            (11, 5):  '1111111111010011',
            (11, 6):  '1111111111010100',
            (11, 7):  '1111111111010101',
            (11, 8):  '1111111111010110',
            (11, 9):  '1111111111010111',
            (11, 10): '1111111111011000',
            (12, 1):  '1111111010',
            (12, 2):  '1111111111011001',
            (12, 3):  '1111111111011010',
            (12, 4):  '1111111111011011',
            (12, 5):  '1111111111011100',
            (12, 6):  '1111111111011101',
            (12, 7):  '1111111111011110',
            (12, 8):  '1111111111011111',
            (12, 9):  '1111111111100000',
            (12, 10): '1111111111100001',
            (13, 1):  '11111111000',
            (13, 2):  '1111111111100010',
            (13, 3):  '1111111111100011',
            (13, 4):  '1111111111100100',
            (13, 5):  '1111111111100101',
            (13, 6):  '1111111111100110',
            (13, 7):  '1111111111100111',
            (13, 8):  '1111111111101000',
            (13, 9):  '1111111111101001',
            (13, 10): '1111111111101010',
            (14, 1):  '1111111111101011',
            (14, 2):  '1111111111101100',
            (14, 3):  '1111111111101101',
            (14, 4):  '1111111111101110',
            (14, 5):  '1111111111101111',
            (14, 6):  '1111111111110000',
            (14, 7):  '1111111111110001',
            (14, 8):  '1111111111110010',
            (14, 9):  '1111111111110011',
            (14, 10): '1111111111110100',
            (15, 0):  '11111111001',
            (15, 1):  '1111111111110101',
            (15, 2):  '1111111111110110',
            (15, 3):  '1111111111110111',
            (15, 4):  '1111111111111000',
            (15, 5):  '1111111111111001',
            (15, 6):  '1111111111111010',
            (15, 7):  '1111111111111011',
            (15, 8):  '1111111111111100',
            (15, 9):  '1111111111111101',
            (15, 10): '1111111111111110',
            # Extra keys for long zero runs
            (16, 1):  'b0000000000000001',
            (17, 1):  'b0000000000000010',
            (18, 1):  'b0000000000000011',
            (19, 1):  'b0000000000000100',
            (20, 1):  'b0000000000000101',
            (21, 1):  'b0000000000000110',
            (22, 1):  'b0000000000000111',
            (23, 1):  'b0000000000001000',
            (24, 1):  'b0000000000001001',

    }

In [0]:
class Encoder(JPEG):
    """Encoding JPEG class."""
    
    def __init__(self, source):
        s = time.time()
        super().__init__()
        assert os.path.isfile(source), 'Incorrect file path.'
        self.source = source
        self.type, self.matrix, self.shape = self.load_file(source)
        assert self.matrix is not None, 'Expected image not to be none. Loading stream error.'
        assert len(self.matrix.shape) > 2, 'Expected 3 channels image. Input image error.'
        self.channels = self.matrix.shape[2]
        self.matrix = cv2.cvtColor(self.matrix, cv2.COLOR_RGB2YCrCb) 
        #self.Y, selfCr, self.Cb = self.matrix[:,:,0], self.matrix[:,:,1], self.matrix[:,:,2]
        if self.chroma_sampling: 
            self.matrix = Encoder.sub_sample(self.matrix, self.chroma_type)
        self.matrix, self.padding = Encoder.pad_matrix(self.matrix, self.block_shape)
        self.nblocks, self.blocks = Encoder.split_matrix(self.matrix, self.block_shape)       
        self.converted = self.convert()
        self.quantized = self.quantize()
        self.DC = self.extract_DC()
        self.AC = self.extract_AC()
        self.
        print(time.time()-s)
        
    def convert(self) -> np.ndarray:
        container = np.empty(self.channels, dtype=np.ndarray)
        for channel in range(self.channels):
            container[channel] = np.empty(self.nblocks, dtype=np.ndarray)
            for index in range(self.nblocks):
                container[channel][index] = Encoder.dct_convert(
                    self.blocks[index][:,:, channel])  
        return container

    
    def quantize(self) -> np.ndarray:
        for channel in range(self.channels):
            if channel in JPEG.LUMINANCE:
                for index in range(self.nblocks):
                    self.converted[channel][index] = Encoder.table_quantize(
                        self.converted[channel][index], self.quality * JPEG.K1)
            elif channel in JPEG.CHROMINANCE:
                for index in range(self.nblocks):
                    self.converted[channel][index] = Encoder.table_quantize(
                        self.converted[channel][index], self.quality * JPEG.K2)
        return self.converted
    
    def extract_DC(self) -> np.ndarray:
        container = np.empty(self.channels * self.nblocks, dtype=np.ndarray)\
            .reshape(self.channels, self.nblocks)
        for channel in range(self.channels):
            for index in range(self.nblocks):
                container[channel][index] = self.converted[channel][index][0].astype(np.uint8)
        return container  
    
    def extract_AC(self) -> np.ndarray:
        container = np.empty(self.channels * self.nblocks * 63, dtype=np.ndarray)\
            .reshape(self.channels, self.nblocks, 63)
        for channel in range(self.channels):
            for index in range(self.nblocks):
                container[channel][index] = self.converted[channel][index].flatten(order='F')\
                    [JPEG.K4.reshape((64),order='F')][1:].astype(np.uint8)
        return container

    @staticmethod
    def load_file(source: str) -> (np.ndarray, np.ndarray):
        """Loads file as an 8 bit symbol stream and classifies it by its extension name."""
        if source.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff', '.tif', '.bmp')):
            output = cv2.imread(source)
            return FileType.IMAGE, output, output.shape
        elif source.lower().endswith(('.txt', '.pdf')):
            output = np.fromfile(source, np.uint8)
            return FileType.TEXT, output, (output.shape[0], 0)
        elif source.lower().endswith('.mp3'):
            output = np.fromfile(source, np.uint8)
            return FileType.SOUND, output, (output.shape[0], 0)
        else:
            raise ValueError('Unspecified encoding for this file type.')
    
    @staticmethod
    def sub_sample(matrix: np.ndarray, chroma_type: tuple) -> np.ndarray:
        """Returns a chroma layer subsample of given matrix."""
        if chroma_type == (4, 2, 2):    
            for index, row in enumerate(matrix): 
                if index % 2 != 0:
                    matrix[:, index] = matrix[:, index-1]
        elif chroma_type == (4, 2, 0):
            for index, row in enumerate(matrix):  
                if index % 2 != 0:
                    matrix[:, index] = matrix[:, index-1]
                    matrix[index, :] = matrix[index-1, :]     
        return matrix
                
    @staticmethod
    def split_matrix(matrix: np.ndarray, block_shape: tuple) -> np.ndarray:
        """Splits a matrix into sub-matrices of given number of rows and columns."""        
        assert np.sum(np.mod(matrix.shape[:2], (block_shape[0], block_shape[1]))) == 0,\
                'Matrix with {} rows and {} columns cannot be subdivided into {}x{} blocks'\
                .format(matrix.shape[0], matrix.shape[1], block_shape[0], block_shape[1])
        nblocks = (matrix.shape[0] * matrix.shape[1]) // (block_shape[0] * block_shape[1])
        return nblocks, np.asarray([np.hsplit(x, matrix.shape[1]//block_shape[1]) 
                           for x in np.vsplit(matrix, matrix.shape[0]//block_shape[0])])\
                           .reshape(nblocks, block_shape[0], block_shape[1], matrix.shape[2])

    @staticmethod
    def pad_matrix(matrix: np.ndarray, block_shape: tuple) -> (np.ndarray, tuple):
        """Pads a matrix with zeros to a block shape modulus and returns padding."""
        rows_pad = ((block_shape[0] - matrix.shape[0] % block_shape[0]) % block_shape[0])
        columns_pad = ((block_shape[1] - matrix.shape[1] % block_shape[1]) % block_shape[1])
        return (np.pad(matrix, ((0, rows_pad), (0, columns_pad), (0, 0)), mode='constant'), 
                (np.unpackbits(np.uint8(rows_pad)), np.unpackbits(np.uint8(columns_pad))))
        
    @staticmethod
    def dct_convert(block: np.ndarray) -> np.ndarray:
        return cv2.dct(block.astype(np.float64))
    
    @staticmethod
    def idct_convert(block: np.ndarray) -> np.ndarray:
        return cv2.idct(block.astype(np.float64))
    
    @staticmethod
    def table_quantize(block: np.ndarray, table: np.ndarray):
        return (block / table).round().astype(int)
    
    
c = Encoder(Notebook.RESOURCES + 'Lena.tiff')


53.203770875930786


In [0]:
# zig-zag order
zigzag = np.zeros((8, 8))
zigzag[0] = [ 0,  1,  5,  6, 14, 15, 27, 28]
zigzag[1] = [ 2,  4,  7, 13, 16, 26, 29, 42]
zigzag[2] = [ 3,  8, 12, 17, 25, 30, 41, 43]
zigzag[3] = [ 9, 11, 18, 24, 31, 40, 44, 53]
zigzag[4] = [10, 19, 23, 32, 39, 45, 52, 54]
zigzag[5] = [20, 22, 33, 38, 46, 51, 55, 60]
zigzag[6] = [21, 34, 37, 47, 50, 56, 59, 61]
zigzag[7] = [35, 36, 48, 49, 57, 58, 62, 63]
#índice para ordenar valores de array 1D em zigzag
ind_O = zigzag.reshape((64),order='F').astype('int')
#índice para repor ordem original de array 1D em zigzag
ind_Z = np.argsort(ind_O)


#Exemplo:
np.random.seed(1)
D1 = np.random.randint(-10,10,size=(8,8))
print(D1)
D2 = D1.flatten(order='F')[ind_Z]
print(D2)
D3 = D2[ind_O].reshape((8,8),order='F')
print(D3)

[[ -5   1   2  -2  -1   1  -5   5]
 [-10   6  -9   2  -3   3  -4   8]
 [ -5   8   1   0   4   8  -6  -1]
 [  7 -10   3  -1  -1  -3  -9 -10]
 [  7  -2   3   9   5   0  -2  -3]
 [ -7  -4   7  -7  -6   7   1   2]
 [  6   3   9  -1   8   5 -10  -6]
 [  5  -8  -3  -2  -1  -7  -3  -6]]
[ -5   1 -10  -5   6   2  -2  -9   8   7   7 -10   1   2  -1   1  -3   0
   3  -2  -7   6  -4   3  -1   4   3  -5   5  -4   8  -1   9   7   3   5
  -8   9  -7   5  -3  -6   8  -1  -9   0  -6  -1  -3  -2   8   7  -2 -10
  -3   1   5  -1  -7 -10   2  -6  -3  -6]
[[ -5   1   2  -2  -1   1  -5   5]
 [-10   6  -9   2  -3   3  -4   8]
 [ -5   8   1   0   4   8  -6  -1]
 [  7 -10   3  -1  -1  -3  -9 -10]
 [  7  -2   3   9   5   0  -2  -3]
 [ -7  -4   7  -7  -6   7   1   2]
 [  6   3   9  -1   8   5 -10  -6]
 [  5  -8  -3  -2  -1  -7  -3  -6]]


In [0]:
d = cv2.imread(Notebook.RESOURCES + 'lena.tiff') 
print(d.shape)
e = cv2.cvtColor(d, cv2.COLOR_RGB2YCrCb)[:, :, 0]  
print(e[:, :, 0].shape)

(512, 512, 3)


IndexError: ignored

In [0]:
class Decoder(JPEG):
    pass

In [0]:
from numpy import abs
from numpy import int
from numpy import binary_repr


class DC:

    def __init__(self, amplitude: int, size: int=0):
        self.__amplitude = amplitude
        self.__size = size

        if self.__amplitude != 0:
            self.__size = len(binary_repr(abs(self.__amplitude)))

    @property
    def amplitude(self):
        return self.__amplitude

    @property
    def size(self):
        return self.__size

    @property
    def elements(self):
        return self.__amplitude

    def __str__(self):
        return "({size}, {amplitude})".format(
            size=self.__size,
            amplitude=self.__amplitude
        )


In [0]:
from numpy import abs
from numpy import binary_repr
from numpy import int
from numpy import ndarray, zeros
from numpy import sum


class AC:

    def __init__(self, zero_run_length: ndarray, amplitude: ndarray):
        self.__zero_run_lengths = zero_run_length
        self.__amplitudes = amplitude
        self.__sizes = zeros(self.__amplitudes.shape[0]).astype(int)

        for i in range(self.__amplitudes.shape[0]):
            bits = binary_repr(abs(self.__amplitudes[i]))
            self.__sizes[i] = int(len(bits))

    @property
    def zero_run_lengths(self):
        return self.__zero_run_lengths

    @property
    def amplitudes(self):
        return self.__amplitudes

    @property
    def elements(self):
        zeros_index = self.__zero_run_lengths > 0
        total_zeros = sum(self.__zero_run_lengths[zeros_index])

        elements = zeros(self.__amplitudes.shape[0] + total_zeros)

        i = 0
        j = 0

        while i < len(elements):
            if self.__zero_run_lengths[j] > 0:
                i += self.__zero_run_lengths[j]

            elements[i] = self.__amplitudes[j]

            j += 1
            i += 1

        return elements

    @property
    def sizes(self):
        return self.__sizes

    def __str__(self):
        text = ""

        for i in range(len(self.__amplitudes)):
            text = "{}{}({zrl}, {size})({amplitude})".format(
                text,
                " " if i == 0 else "",
                zrl=self.__zero_run_lengths[i],
                size=self.__sizes[i],
                amplitude=self.__amplitudes[i]
            )

        return text


In [0]:
from numpy import abs, int
from numpy import binary_repr
from numpy import zeros


class Stream:

    def __init__(self, pretty=False):
        self.__regular_text = ""
        self.__pretty_text = ""
        self.__enable_pretty = pretty

    @property
    def regular(self):
        return self.__regular_text

    @property
    def pretty(self):
        return self.__pretty_text

    def join(self, stream):
        self.__regular_text = "{}{}".format(
            self.__regular_text,
            stream.regular
        )

        if not self.__enable_pretty:
            return self

        self.__pretty_text = "{this}{space}{new}".format(
            this=self.__pretty_text,
            space=" " if self.__pretty_text != "" else "",
            new=stream.pretty
        )

        return self

    def add(self, size_bits: str, signal_bits: str, amplitude_bits: str):
        if amplitude_bits == "0":
            self.__regular_text = "{stream}{size}".format(
                stream=self.__regular_text,
                size=size_bits
            )
        else:
            self.__regular_text = "{stream}{size}{signal}{amplitude}".format(
                stream=self.__regular_text,
                size=size_bits,
                signal=signal_bits,
                amplitude=amplitude_bits
            )

        if not self.__enable_pretty:
            return

        if amplitude_bits == "0":
            self.__pretty_text = "{stream}{space}{size}".format(
                stream=self.__pretty_text,
                space=" " if self.__pretty_text != "" else "",
                size=size_bits
            )
        else:
            self.__pretty_text = "{stream}{space}{size} {signal} {amplitude}".format(
                stream=self.__pretty_text,
                space=" " if self.__pretty_text != "" else "",
                size=size_bits,
                signal=signal_bits,
                amplitude=amplitude_bits
            )

    def add_prefix(self, prefix):
        self.__regular_text = "{prefix}{stream}".format(
            prefix=prefix,
            stream=self.__regular_text
        )

        if not self.__enable_pretty:
            return

        self.__pretty_text = "{prefix}{space}{stream}".format(
            prefix=prefix,
            space=" " if self.__pretty_text != "" else "",
            stream=self.__pretty_text
        )

    def add_suffix(self, suffix: str):
        self.__regular_text = "{stream}{suffix}".format(
            stream=self.__regular_text,
            suffix=suffix
        )

        if not self.__enable_pretty:
            return

        self.__pretty_text = "{stream} {suffix}".format(
            stream=self.__pretty_text,
            suffix=suffix
        )

    def remove(self, size: int):
        self.__regular_text = self.__regular_text[size:]

        if not self.__enable_pretty:
            return

        if self.__pretty_text is None:
            return

        total_spaces = 0

        for c in range(len(self.__pretty_text[0: size + 3])):
            if self.__pretty_text[c] == " ":
                total_spaces += 1

        self.__pretty_text = self.__pretty_text[size + total_spaces:]

    def __str__(self):
        if self.__pretty_text != "":
            return self.__pretty_text

        return self.__regular_text

    def __len__(self):
        return len(self.__regular_text)

    def __getitem__(self, item):
        return self.__regular_text[item]

eob = "1010"

k3 = {
    0: "00",
    1: "010",
    2: "011",
    3: "100",
    4: "101",
    5: "110",
    6: "1110",
    7: "11110",
    8: "111110",
    9: "1111110",
    10: "11111110",
    11: "111111110",
}

k5 = {
    # (0, 0): "1010",
    (0, 1): "00",
    (0, 2): "01",
    (0, 3): "100",
    (0, 4): "1011",
    (0, 5): "11010",
    (0, 6): "1111000",
    (0, 7): "11111000",
    (0, 8): "1111110110",
    (0, 9): "1111111110000010",
    (0, 10): "1111111110000011",
    (1, 1): "1100",
    (1, 2): "11011",
    (1, 3): "1111001",
    (1, 4): "111110110",
    (1, 5): "11111110110",
    (1, 6): "1111111110000100",
    (1, 7): "1111111110000101",
    (1, 8): "1111111110000110",
    (1, 9): "1111111110000111",
    (1, 10): "1111111110001000",
    (2, 1): "11100",
    (2, 2): "11111001",
    (2, 3): "1111110111",
    (2, 4): "111111110100",
    (2, 5): "1111111110001001",
    (2, 6): "1111111110001010",
    (2, 7): "1111111110001011",
    (2, 8): "1111111110001100",
    (2, 9): "1111111110001101",
    (2, 10): "1111111110001110",
    (3, 1): "111010",
    (3, 2): "111110111",
    (3, 3): "111111110101",
    (3, 4): "1111111110001111",
    (3, 5): "1111111110010000",
    (3, 6): "1111111110010001",
    (3, 7): "1111111110010010",
    (3, 8): "1111111110010011",
    (3, 9): "1111111110010100",
    (3, 10): "1111111110010101",
    (4, 1): "111011",
    (4, 2): "1111111000",
    (4, 3): "1111111110010110",
    (4, 4): "1111111110010111",
    (4, 5): "1111111110011000",
    (4, 6): "1111111110011001",
    (4, 7): "1111111110011010",
    (4, 8): "1111111110011011",
    (4, 9): "1111111110011100",
    (4, 10): "1111111110011101",
    (5, 1): "1111010",
    (5, 2): "11111110111",
    (5, 3): "1111111110011110",
    (5, 4): "1111111110011111",
    (5, 5): "1111111110100000",
    (5, 6): "1111111110100001",
    (5, 7): "1111111110100010",
    (5, 8): "1111111110100011",
    (5, 9): "1111111110100100",
    (5, 10): "1111111110100101",
    (6, 1): "1111011",
    (6, 2): "111111110110",
    (6, 3): "1111111110100110",
    (6, 4): "1111111110100111",
    (6, 5): "1111111110101000",
    (6, 6): "1111111110101001",
    (6, 7): "1111111110101010",
    (6, 8): "1111111110101011",
    (6, 9): "1111111110101100",
    (6, 10): "1111111110101101",
    (7, 1): "11111010",
    (7, 2): "111111110111",
    (7, 3): "1111111110101110",
    (7, 4): "1111111110101111",
    (7, 5): "1111111110110000",
    (7, 6): "1111111110110001",
    (7, 7): "1111111110110010",
    (7, 8): "1111111110110011",
    (7, 9): "1111111110110100",
    (7, 10): "1111111110110101",
    (8, 1): "111111000",
    (8, 2): "111111111000000",
    (8, 3): "1111111110110110",
    (8, 4): "1111111110110111",
    (8, 5): "1111111110111000",
    (8, 6): "1111111110111001",
    (8, 7): "1111111110111010",
    (8, 8): "1111111110111011",
    (8, 9): "1111111110111100",
    (8, 10): "1111111110111101",
    (9, 1): "111111001",
    (9, 2): "1111111110111110",
    (9, 3): "1111111110111111",
    (9, 4): "1111111111000000",
    (9, 5): "1111111111000001",
    (9, 6): "1111111111000010",
    (9, 7): "1111111111000011",
    (9, 8): "1111111111000100",
    (9, 9): "1111111111000101",
    (9, 10): "1111111111000110",
    (10, 1): "111111010",
    (10, 2): "1111111111000111",
    (10, 3): "1111111111001000",
    (10, 4): "1111111111001001",
    (10, 5): "1111111111001010",
    (10, 6): "1111111111001011",
    (10, 7): "1111111111001100",
    (10, 8): "1111111111001101",
    (10, 9): "1111111111001110",
    (10, 10): "1111111111001111",
    (11, 1): "1111111001",
    (11, 2): "1111111111010000",
    (11, 3): "1111111111010001",
    (11, 4): "1111111111010010",
    (11, 5): "1111111111010011",
    (11, 6): "1111111111010100",
    (11, 7): "1111111111010101",
    (11, 8): "1111111111010110",
    (11, 9): "1111111111010111",
    (11, 10): "1111111111011000",
    (12, 1): "1111111010",
    (12, 2): "1111111111011001",
    (12, 3): "1111111111011010",
    (12, 4): "1111111111011011",
    (12, 5): "1111111111011100",
    (12, 6): "1111111111011101",
    (12, 7): "1111111111011110",
    (12, 8): "1111111111011111",
    (12, 9): "1111111111100000",
    (12, 10): "1111111111100001",
    (13, 1): "11111111000",
    (13, 2): "1111111111100010",
    (13, 3): "1111111111100011",
    (13, 4): "1111111111100100",
    (13, 5): "1111111111100101",
    (13, 6): "1111111111100110",
    (13, 7): "1111111111100111",
    (13, 8): "1111111111101000",
    (13, 9): "1111111111101001",
    (13, 10): "1111111111101010",
    (14, 1): "1111111111101011",
    (14, 2): "1111111111101100",
    (14, 3): "1111111111101101",
    (14, 4): "1111111111101110",
    (14, 5): "1111111111101111",
    (14, 6): "1111111111110000",
    (14, 7): "1111111111110001",
    (14, 8): "1111111111110010",
    (14, 9): "1111111111110011",
    (14, 10): "1111111111110100",
    (15, 0): "11111111001",
    (15, 1): "1111111111110101",
    (15, 2): "1111111111110110",
    (15, 3): "1111111111110111",
    (15, 4): "1111111111111000",
    (15, 5): "1111111111111001",
    (15, 6): "1111111111111010",
    (15, 7): "1111111111111011",
    (15, 8): "1111111111111100",
    (15, 9): "1111111111111101",
    (15, 10): "1111111111111110",
    # Extra keys
    (16, 1): "b0000000000000001",
    (17, 1): "b0000000000000010",
    (18, 1): "b0000000000000011",
    (19, 1): "b0000000000000100",
    (20, 1): "b0000000000000101",
    (21, 1): "b0000000000000110",
    (22, 1): "b0000000000000111",
    (23, 1): "b0000000000001000",
    (24, 1): "b0000000000001001",

}


def generate_signal_bits(amplitude: int) -> str:
    return "1" if amplitude < 0 else "0"


def generate_signal(bit: str) -> int:
    return 1 if bit == "0" else -1


def generate_amplitude_bits(amplitude: int) -> str:
    return binary_repr(abs(amplitude))


def encode(dc: DC, ac: AC) -> Stream:
    stream = Stream()

    # DC stream
    size_bits = k3.get(dc.size)
    signal_bits = generate_signal_bits(dc.amplitude)
    amplitude_bits = generate_amplitude_bits(dc.amplitude)

    stream.add(size_bits, signal_bits, amplitude_bits)

    # AC stream
    amplitudes = ac.amplitudes
    zrls = ac.zero_run_lengths
    sizes = ac.sizes

    for i in range(len(amplitudes)):

        if zrls[i] > 15:
            n_zeros = zrls[i]
            while n_zeros > 0:
                if n_zeros > 15:
                    size_bits = k5.get((15, 0))
                    signal_bits = generate_signal_bits(0)
                    amplitude_bits = generate_amplitude_bits(0)
                else:
                    size_bits = k5.get((n_zeros, sizes[i]))
                    signal_bits = generate_signal_bits(amplitudes[i])
                    amplitude_bits = generate_amplitude_bits(amplitudes[i])

                stream.add(size_bits, signal_bits, amplitude_bits)
                n_zeros -= 15
        else:
            size_bits = k5.get((zrls[i], sizes[i]))
            if size_bits is None:
                size_bits = k5.get((0, 1))
            signal_bits = generate_signal_bits(amplitudes[i])
            amplitude_bits = generate_amplitude_bits(amplitudes[i])

            stream.add(size_bits, signal_bits, amplitude_bits)

    # EOB stream
    stream.add_suffix(eob)

    return stream


def decode(stream: Stream, total_elements: int) -> (DC, AC, Stream):
    dc = None
    ac = None

    i = 0

    # DC stream
    while i < len(stream) and dc is None:
        i += 1

        for k in k3:
            if stream[0: i] != k3.get(k):
                continue

            size = k
            amplitude = 0

            if size > 0:
                signal = generate_signal(stream[i: i + 1])
                i += 1
                amplitude = int(stream[i: i + size], 2) * signal

            dc = DC(amplitude, size)
            stream.remove(i + size)
            break

    # AC stream
    zrls = zeros(total_elements, dtype=int)
    amplitudes = zeros(total_elements, dtype=int)
    i = 0
    j = 0

    while stream[0: i] != eob:
        i += 1

        for k in k5:
            if stream[0: i] != k5.get(k):
                continue

            zrls[j], size = k[0], k[1]

            if size > 0:
                signal = generate_signal(stream[i: i + 1])
                i += 1
                amplitudes[j] = int(stream[i: i + size], 2) * signal

            stream.remove(i + size)
            j += 1
            i = 0
            break

    idx = amplitudes != 0
    amplitudes = amplitudes[idx]
    zrls = zrls[idx]

    ac = AC(zrls, amplitudes)

    # EOB stream
    stream.remove(len(eob))

    return dc, ac, stream



In [0]:
import numpy as np
import copy
import cv2
import matplotlib.pyplot as plt
from .entropic_coding import Stream
from . import dct
from . import quantification
from . import ycc
from .block import Block


class JPEG:

    header_size = 16 * 2
    total_layers = 3

    def __init__(self, image: np.ndarray, quality: int = 50):
        self.__raw_image = np.copy(image)
        self.__image = np.copy(image)
        self.__quality = quality
        self.__stream = Stream()
        self.__header = Stream()

    @classmethod
    def from_stream(cls, stream: str):
        cls.__header, cls.__stream = cls.__decode_header(stream)

        return cls

    def __crop_image(self):
        total_lines, total_columns, layers = self.__image.shape

        excess_lines = total_lines % Block.size
        excess_columns = total_columns % Block.size

        for i in range(layers):
            self.__image[:, :, i] = self.__image[
                                    0: total_lines - excess_lines,
                                    0: total_columns - excess_columns, i
                                ]

    def __apply_subsample(self, image: np.array, horizontal_ratio: int, vertical_ratio: int, sample_size: int = 4):
        rows, columns = image.shape
        x = 0
        y = 0
        for row in range(rows):
            for col in range(columns):
                if x < rows and y < columns:
                    sub_matrix = np.ones((1, horizontal_ratio, vertical_ratio)) * image[x][y]
                    image = self.__apply_block(image, sub_matrix, x, y, vertical_ratio)
                    y += vertical_ratio
            x += horizontal_ratio
            y = 0

        return image

    def __apply_block(self, matrix, block, line, col, dim):
        matrix[line: line + dim, col: col + dim] = block
        return matrix

    def __make_subsample(self, horizontal_ratio: int, vertical_ratio: int):
        for frame in range(1, 3):
            self.__image[:, :, frame] = self.__apply_subsample(self.__image[:, :, frame], horizontal_ratio, vertical_ratio)

    def encode(self, horizontal_ratio: int, vertical_ratio: int):
        self.__crop_image()
        if horizontal_ratio != 1:
            self.__make_subsample(horizontal_ratio, vertical_ratio)
        self.__image = ycc.encode(self.__image)

        for i in range(JPEG.total_layers):
            blocks = self.__make_blocks(self.__image[:, :, i])
            previous_block = None

            if i == 0:
                self.__encode_header(blocks.shape[0], blocks.shape[1])

            for row in range(blocks.shape[0]):
                for column in range(blocks.shape[1]):
                    block = dct.encode(blocks[row, column])
                    block = quantification.encode(blocks[row, column], self.__quality)

                    block.encode(previous_block)
                    self.__stream.join(block.stream)

                    previous_block = copy.deepcopy(block)

    def decode(self):

        if self.__header == "":
            return

        if self.__stream == "":
            return

        stream = copy.deepcopy(self.__stream)
        rows = int(self.__header[0: int(JPEG.header_size / 2)], 2)
        columns = int(self.__header[int(JPEG.header_size / 2): JPEG.header_size], 2)

        image = np.zeros(
            (columns * Block.size, rows * Block.size, JPEG.total_layers),
            dtype=np.uint8
        )

        for i in range(JPEG.total_layers):
            previous_block = None

            for row in range(rows):
                for column in range(columns):
                    block, stream = Block.decode(previous_block, stream)
                    previous_block = copy.deepcopy(block)

                    block = quantification.decode(block)
                    block = dct.decode(block)

                    image[
                        row * Block.size: row * Block.size + Block.size,
                        column * Block.size: column * Block.size + Block.size,
                        i,
                    ] = block.elements

        image = ycc.decode(image)
        self.__image = image

    def __encode_header(self, rows: int, columns: int):
        row_bits = np.binary_repr(rows).rjust(np.int(JPEG.header_size / 2), "0")
        column_bits = np.binary_repr(columns).rjust(np.int(JPEG.header_size / 2), "0")

        self.__header.add_prefix(row_bits)
        self.__header.add_prefix(column_bits)

    @staticmethod
    def __decode_header(stream: str) -> (Stream, int, int, str):
        header = Stream()
        header.add_prefix(stream[0: np.int(JPEG.header_size)])

        stream = stream[JPEG.header_size:]

        return header, stream

    def __subtract_128(self):
        self.__image = self.__image - 128

    def __add_128(self):
        self.__image = self.__image + 128

    def __make_blocks(self, image_layer: np.ndarray):
        # Calculate the total number of blocks
        rows = image_layer.shape[0]
        columns = image_layer.shape[1]

        total_blocks_horizontal = np.int(columns / Block.size)
        total_blocks_vertical = np.int(rows / Block.size)
        total_blocks = total_blocks_horizontal * total_blocks_vertical

        # Initialize a bi-dimensional array of blocks
        blocks = [Block] * total_blocks
        blocks = np.reshape(blocks, (total_blocks_horizontal, total_blocks_vertical))

        # Make each block
        for blocks_row in range(blocks.shape[0]):
            for blocks_column in range(blocks.shape[1]):
                elements_row = blocks_row * Block.size
                elements_column = blocks_column * Block.size
                elements = image_layer[
                               elements_row: elements_row + Block.size,
                               elements_column: elements_column + Block.size
                           ]

                blocks[blocks_row, blocks_column] = Block(elements)

        return blocks

    @property
    def stream(self):
        if self.__header is None or self.__stream is None:
            self.encode()

        final_stream = Stream()
        final_stream.join(self.__header).join(self.__stream)

        return final_stream.regular

    @property
    def image(self):
        return self.__image


In [0]:
from numpy import copy
from numpy import ndarray

from . import dpcm
from . import entropic_coding
from . import rlc
from .entropic_coding import Stream


class Block:

    size = 8

    def __init__(self, elements: ndarray):
        self.__elements = copy(elements)
        self.__dc = None
        self.__ac = None
        self.__stream = Stream()

    def __getitem__(self, item):
        return self.__elements[item]

    def __setitem__(self, key, value):
        self.__elements[key] = value

    def __str__(self):
        return self.__elements.__str__()

    @property
    def elements(self):
        return copy(self.__elements)

    @elements.setter
    def elements(self, value):
        self.__elements = value

    @property
    def dc(self):
        return self.__dc

    @dc.setter
    def dc(self, value):
        self.__dc = value

    @property
    def ac(self):
        return self.__ac

    @ac.setter
    def ac(self, value):
        self.__ac = value

    @property
    def stream(self):
        return self.__stream

    def encode(self, previous_block) -> Stream:
        self.__dc = dpcm.encode(previous_block, self)
        self.__ac = rlc.encode(self)
        self.__stream = entropic_coding.encode(self.__dc, self.__ac)

        return self.__stream

    @staticmethod
    def decode(previous_block, stream: Stream):
        dc, ac, stream = entropic_coding.decode(stream, Block.size * Block.size)

        if dc is None or ac is None:
            return None

        elements = rlc.decode(ac, Block.size * Block.size)
        elements[0, 0] = dpcm.decode(previous_block, dc)

        block = Block(elements)

        block.dc = dc
        block.ac = ac

        return block, stream


ImportError: ignored

In [0]:
from cv2 import dct
from cv2 import idct

from .block import Block

### .dct ###

def encode(block: Block) -> Block:
    block.elements = dct(block.elements * 1.0)
    return block


def decode(block: Block) -> Block:
    block.elements = idct(block.elements * 1.0)
    return block


ModuleNotFoundError: ignored

In [0]:
import numpy as np



### .quantification ###

def quality_factor(q: float) -> float:

    if q >= 100:
        return 1

    if q <= 50:
        return q / 50

    return 2 - (q * 2) / 100

print(quality_factor(10))

# k1
def get_matrix() -> np.array:
    return np.array([
        [16, 11, 10, 16, 24, 40, 51, 61],
        [12, 12, 14, 19, 26, 58, 60, 55],
        [14, 13, 16, 24, 40, 57, 69, 56],
        [14, 17, 22, 29, 51, 87, 80, 62],
        [18, 22, 37, 56, 68, 109, 103, 77],
        [24, 35, 55, 64, 81, 104, 113, 92],
        [49, 64, 78, 87, 103, 121, 120, 101],
        [72, 92, 95, 98, 112, 100, 103, 99]
    ])


def encode(block: Block, quality: float=50) -> Block:
    k = get_matrix()
    block.elements = np.round(block.elements / (k * quality_factor(quality))).astype(int)
    return block


def decode(block: Block, quality: float=50) -> Block:
    k = get_matrix()
    block.elements = block.elements * (k * quality_factor(quality))
    return block



0.2


NameError: ignored